In [2]:
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)


df = pd.read_csv('../../bgd.csv', index_col=0)
df = df[df['english_translation'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records')
df

,location,english_translation,notes
sno,,,
1,Arjuna's Vishada Yoga Chapter 1 Verse 1.1,"Dhrtarashtra asked of Sanjaya: O SANJAYA, what...",Arjuna's Vishada Yoga Chapter 1 Verse 1.1 - Dh...
2,Arjuna's Vishada Yoga Chapter 1 Verse 1.2,Sanjaya explained: Now seeing that the army of...,Arjuna's Vishada Yoga Chapter 1 Verse 1.2 - Sa...
3,Arjuna's Vishada Yoga Chapter 1 Verse 1.3,"Behold O, Master, the mighty army of the sons ...",Arjuna's Vishada Yoga Chapter 1 Verse 1.3 - Be...
4,Arjuna's Vishada Yoga Chapter 1 Verse 1.4,"Present here are the mighty archers, peers or ...",Arjuna's Vishada Yoga Chapter 1 Verse 1.4 - Pr...
5,Arjuna's Vishada Yoga Chapter 1 Verse 1.5,"Dhrishtaketu, Chekitana, and the valiant king ...",Arjuna's Vishada Yoga Chapter 1 Verse 1.5 - Dh...
...,...,...,...
696,Moksha-Sanyasa Yoga Chapter 18 Verse 18.74,"Sanjaya concluded: Thus my dear King, I have r...",Moksha-Sanyasa Yoga Chapter 18 Verse 18.74 - S...
697,Moksha-Sanyasa Yoga Chapter 18 Verse 18.75,"By the kindness of the Holy poet, Ved O’Vyas, ...",Moksha-Sanyasa Yoga Chapter 18 Verse 18.75 - B...
698,Moksha-Sanyasa Yoga Chapter 18 Verse 18.76,"The more I recall these glorious words, my Kin...",Moksha-Sanyasa Yoga Chapter 18 Verse 18.76 - T...


In [3]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [5]:
encoder = SentenceTransformer('dunzhang/stella_en_1.5B_v5') # Model to create embeddings

model.safetensors:  62%|######2   | 3.85G/6.17G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

In [ ]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="bhakti",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/sz/3wq8394j4g56kkvsx4sg91v80000gn/T/ipykernel_2831/843202155.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [6]:
sliding_window_length = 5
sliding_window_step = 2
data2 = []

for i in range(0, len(data)-sliding_window_length+1, sliding_window_step):
    window = []
    for j in range(sliding_window_length):
        window.append(data[i+j]["notes"])
    window = " ".join(window)
    data2.append(dict(notes=window))




In [7]:
# vectorize!
qdrant.upload_points(
    collection_name="bhakti",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data2) # data is the variable holding all the sick one liners
    ]
)

In [21]:
user_prompt = "But those who worship the formless, the Indefinable, the Imperishable, the Unmanifest, the Omnipresent, the Unthinkable, the Unchangeable, the Immovable, the External- Having controlled all of the senses, always evenminded, thinking of the welfare of all beings, they also come to Me."

In [22]:
# Search time for some real spirituality!

hits = qdrant.search(
    collection_name="bhakti",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  #print(hit.payload, "score:", hit.score)
    pp.pprint(hit.payload)

{ 'notes': 'Bhakti Yoga Chapter 12 Verse 12.3 - But those who worship the '
           'formless, the Indefinable, the Imperishable, the Unmanifest, the '
           'Omnipresent, the Unthinkable, the Unchangeable, the Immovable, the '
           'External- Having controlled all of the senses, always evenminded, '
           'thinking of the welfare of all beings, they also come to Me. '
           'Bhakti Yoga Chapter 12 Verse 12.4 - But those who worship the '
           'formless, the Indefinable, the Imperishable, the Unmanifest, the '
           'Omnipresent, the Unthinkable, the Unchangeable, the Immovable, the '
           'External- Having controlled all of the senses, always evenminded, '
           'thinking of the welfare of all beings, they also come to Me. '
           'Bhakti Yoga Chapter 12 Verse 12.5 - However, the worship of God '
           'without form is very difficult for the embodied because it is very '
           'hard to set the mind on the unmanifest, i.e. th

In [23]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [24]:
reference_notes = [search_result['notes'] for search_result in search_results]
reference_notes

['Bhakti Yoga Chapter 12 Verse 12.3 - But those who worship the formless, the Indefinable, the Imperishable, the Unmanifest, the Omnipresent, the Unthinkable, the Unchangeable, the Immovable, the External- Having controlled all of the senses, always evenminded, thinking of the welfare of all beings, they also come to Me. Bhakti Yoga Chapter 12 Verse 12.4 - But those who worship the formless, the Indefinable, the Imperishable, the Unmanifest, the Omnipresent, the Unthinkable, the Unchangeable, the Immovable, the External- Having controlled all of the senses, always evenminded, thinking of the welfare of all beings, they also come to Me. Bhakti Yoga Chapter 12 Verse 12.5 - However, the worship of God without form is very difficult for the embodied because it is very hard to set the mind on the unmanifest, i.e. the one not capable of being recognized with the aid of the senses. Bhakti Yoga Chapter 12 Verse 12.6 - But those who worship Me, dedicating all actions to Me, regarding Me as the 

In [25]:
"""You are an expert in ancient Indian History and especially the Bhagavad Gita. Use the following references from the Gita to answer my question at the end.
    Reference = """+str(reference_notes)+""" Here is the question: """+user_prompt

"You are an expert in ancient Indian History and especially the Bhagavad Gita. Use the following references from the Gita to answer my question at the end.\n    Reference = ['Bhakti Yoga Chapter 12 Verse 12.3 - But those who worship the formless, the Indefinable, the Imperishable, the Unmanifest, the Omnipresent, the Unthinkable, the Unchangeable, the Immovable, the External- Having controlled all of the senses, always evenminded, thinking of the welfare of all beings, they also come to Me. Bhakti Yoga Chapter 12 Verse 12.4 - But those who worship the formless, the Indefinable, the Imperishable, the Unmanifest, the Omnipresent, the Unthinkable, the Unchangeable, the Immovable, the External- Having controlled all of the senses, always evenminded, thinking of the welfare of all beings, they also come to Me. Bhakti Yoga Chapter 12 Verse 12.5 - However, the worship of God without form is very difficult for the embodied because it is very hard to set the mind on the unmanifest, i.e. the one

In [26]:
# Now time to connect to the local large language model
# from openai import OpenAI
# client = OpenAI(
#     base_url="http://localhost:11434/api/generate", # "http://<Your api-server IP>:port"
#     api_key = "sk-no-key-required"
# )
# completion = client.chat.completions.create(
#     model="LLaMA_CPP",
#     messages=[
#         {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
#         {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
#         {"role": "assistant", "content": str(search_results)}
#     ]
# )
# print(completion.choices[0].message)

import requests

url = "http://localhost:11434/api/generate"
payload = {
    "model": "llama2",
    "prompt":"""You are an expert in ancient Indian History and especially the Bhagavad Gita. ONLY Use the following references from the Gita to answer my question at the end. DO NOT use any prior knowledge. If you are not sure, please say "Source knowledge is insufficient and do not elaborate any further.
    Reference = """+str(reference_notes)+""" Here is the question: """+user_prompt,
    "stream":False
}

x = requests.post(url, json = payload)

print(x.json()['response'])

The reference you provided from the Bhagavad Gita contains verses 12.3-5 and 12.7-12.26. To answer your question, we need to analyze these verses in the context of the Gita's teachings on worship and spiritual practice.

Verse 12.3 states that those who worship the formless, indefinable, imperishable, unmanifest, omnipresent, unthinkable, unchangeable, immovable, external God with controlled senses, always even-minded, thinking of the welfare of all beings, come to God. This verse suggests that worshiping God in a state of inner peace, control over the senses, and selflessness leads to spiritual growth and union with God.

Verse 12.5 notes that the worship of God without form is difficult for embodied beings because it is hard to set the mind on the unmanifest, i.e., the one not capable of being recognized with the aid of the senses. This verse highlights the challenge of attaining God beyond the limitations of the material world.

Verse 12.7 states that those who worship Me, dedicatin